<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## Importing Libraries

In [54]:
import pandas as pd
import numpy as np
import sqlalchemy as db

## Dataset

In [55]:
# Load the data
bank_df = pd.read_csv('bank_marketing.csv')

## Introductory Details

In [56]:
bank_df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [57]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [58]:
bank_df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [59]:
bank_df.shape

(41188, 22)

## Data Cleaning

### Splitting DataFrame

In [60]:
# Splitting DataFrame

client = bank_df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

campaign = bank_df[['client_id', 'campaign', 'month', 'day_of_week', 'duration', 'pdays', 'previous', 'poutcome', 'y']]

economics = bank_df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

### Rename, clean, create, and delete columns

In [61]:
# Renaming columns

client = client.rename(columns={'client_id': 'id'})

campaign = campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign': 'number_contacts'})

economics = economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'})

# Cleaning columns

client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)
client['job'] = client['job'].str.replace('.', '')
client['credit_default'] = client['credit_default'].replace('unknown', np.nan)

campaign['campaign_outcome']= campaign['campaign_outcome'].replace('no', '0').replace('yes', '1')
campaign['previous_outcome']= campaign['previous_outcome'].replace('nonexistent', np.nan).replace('failure', '0').replace('success', '1')

# Creating new columns
campaign['campaign_id'] = 1

campaign['month'] = campaign['month'].str.capitalize()
campaign['day_of_week'] = campaign['day_of_week'].str.capitalize()
campaign['year'] = '2022'

campaign['last_contact_date'] = campaign['year']  + '-' + campaign['month'] + '-' + campaign['day_of_week']

campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format='%Y-%b-%a')

# Deleting columns
campaign = campaign.drop(['month', 'day_of_week', 'year'], axis=1)

# Shift column 'campaign_id' to first position
first_column = campaign.pop('campaign_id')
campaign.insert(0, 'id', first_column)

# Store the DataFrames
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

## Cleaned Datasets

### Client Data

In [62]:
print(client.head())

   id  age        job  marital    education credit_default housing loan
0   0   56  housemaid  married     basic_4y             no      no   no
1   1   57   services  married  high_school            NaN      no   no
2   2   37   services  married  high_school             no     yes   no
3   3   40      admin  married     basic_6y             no      no   no
4   4   56   services  married  high_school             no      no  yes


### Campaign Data

In [63]:
print(campaign.head())

   id  client_id  ...  campaign_outcome  last_contact_date
0   1          0  ...                 0         2022-05-01
1   1          1  ...                 0         2022-05-01
2   1          2  ...                 0         2022-05-01
3   1          3  ...                 0         2022-05-01
4   1          4  ...                 0         2022-05-01

[5 rows x 9 columns]


### Economics Data

In [64]:
print(economics.head())

   client_id  emp_var_rate  ...  euribor_three_months  number_employed
0          0           1.1  ...                 4.857           5191.0
1          1           1.1  ...                 4.857           5191.0
2          2           1.1  ...                 4.857           5191.0
3          3           1.1  ...                 4.857           5191.0
4          4           1.1  ...                 4.857           5191.0

[5 rows x 5 columns]


## Add to Database

In [65]:
engine = db.create_engine("sqlite:///bank_database.sqlite")

client.to_sql('client_data', engine)
campaign.to_sql('campaign_data', engine)
economics.to_sql('economics_data', engine)

41188

### Pull Data from newly created database

In [66]:
with engine.connect() as conn:
    output = conn.execute("SELECT * FROM client_data LIMIT 5")
    print(output.fetchall())



[(0, 0, 56, 'housemaid', 'married', 'basic_4y', 'no', 'no', 'no'), (1, 1, 57, 'services', 'married', 'high_school', None, 'no', 'no'), (2, 2, 37, 'services', 'married', 'high_school', 'no', 'yes', 'no'), (3, 3, 40, 'admin', 'married', 'basic_6y', 'no', 'no', 'no'), (4, 4, 56, 'services', 'married', 'high_school', 'no', 'no', 'yes')]
